In [63]:
from __future__ import print_function    # (at top of module)
import warnings
#warnings.filterwarnings('always')
from spotipy.oauth2 import SpotifyClientCredentials
import json
import spotipy
import time
import sys
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]
from matplotlib.pyplot import figure
import math
import seaborn as sns
import config

In [64]:
# Read the data from the file
data = pd.read_csv('data_500_entries_youtube.csv')
print("Number of entries in original data: " + str(len(data.index)))
data.head()

Number of entries in original data: 570


,song_id,song_title,artist,popularity,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,youtube_video_title
0,spotify:track:5ygDXis42ncn6kYG14lEVG,Baby Shark,[Pinkfong],77,0.840,0.3410,115.062,0.2270,0.2450,0.000000,4,0.825,7,96333,-3.651,0.520,1,1956582159,Baby Shark Dance | Sing and Dance! | Animal So...
1,spotify:track:7fa9MBXhVfQ8P8Df9OEbD8,Girls Like You (feat. Cardi B),"[Maroon 5, Cardi B]",86,0.541,0.1300,124.959,0.0505,0.5680,0.000000,4,0.851,0,235545,-6.825,0.448,1,1300452389,Maroon 5 - Girls Like You ft. Cardi B
2,spotify:track:6De0lHrwBfPfrhorm9q1Xl,Me Rehúso,[Danny Ocean],83,0.804,0.0494,104.823,0.0677,0.0231,0.000000,4,0.744,1,205715,-6.327,0.426,1,1229501096,Danny Ocean - Me Rehúso (Official Audio)
3,spotify:track:1j6xOGusnyXq3l6IryKF3G,Déjala Que Vuelva (feat. Manuel Turizo),"[Piso 21, Manuel Turizo]",74,0.788,0.0753,170.019,0.0785,0.0482,0.000000,4,0.681,1,220117,-4.323,0.839,1,1216075058,Piso 21 - Déjala Que Vuelva (feat. Manuel Turi...
4,spotify:track:2ijef6ni2amuunRoKTlgww,Sin Pijama,"[Becky G, Natti Natasha]",90,0.745,0.1040,94.014,0.0464,0.3540,0.000029,4,0.791,11,188560,-3.695,0.820,0,1071141995,Becky G Natti Natasha - Sin Pijama (Video Ofic...


In [65]:
if 'song_id' in data.columns:
    data = data.drop_duplicates(subset=['song_id'], keep='first')
else:
    data = data.drop_duplicates(subset=['song_title'], keep='first')
    
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 570


In [66]:
data = data[data.popularity > 50]
print("Number of entries in original data after cleaning: " + str(len(data.index)))

Number of entries in original data after cleaning: 570


In [67]:
from project_modules import *
# Make a copy of the data to which we will ad labels and then remove any 
# columns that we will not need
# This is currently a duplicate of the functionality above - could maybe only do this in one place

final_data = label_data(data, 90)

# Drop unnecessary columns from original data
if 'song_id' in data.columns:
    final_data.drop(['song_id', 'song_title', 'artist', 'popularity', 'youtube_video_title'], 1, inplace=True)
else:
    final_data.drop(['song_title', 'artist', 'popularity'], 1, inplace=True) 


Number of popular examples after thresholding :  47
Number of not popular examples after thresholding :  523


In [68]:
from sklearn.preprocessing import scale, MinMaxScaler

COLUMNS_TO_SCALE = ["energy", "liveness", "tempo", 
                    "speechiness", "acousticness", "instrumentalness", 
                    "time_signature", "danceability", "key", 
                    "duration", "loudness", "valence", "mode", "youtube_view_count"]
# Keep data in a temp variable for testing
scaled_data = scale_data_normalize(final_data, COLUMNS_TO_SCALE)

# Copy data back
final_data = scaled_data.copy()

#Just to check that everything is fine
final_data.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode,youtube_view_count,is_popular
0,0.838293,0.331118,0.360693,0.399286,0.247690,0.000000,0.75,0.789030,0.636364,0.016514,0.807462,0.522258,1.0,1.000000,0
1,0.502527,0.111965,0.431906,0.049366,0.574298,0.000000,0.75,0.825598,0.000000,0.442256,0.594084,0.444083,1.0,0.664655,0
2,0.797866,0.028251,0.287019,0.083466,0.023312,0.000000,0.75,0.675105,0.090909,0.351029,0.627563,0.420195,1.0,0.628392,0
3,0.779899,0.055152,0.756132,0.104877,0.048692,0.000000,0.75,0.586498,0.090909,0.395074,0.762286,0.868621,1.0,0.621530,0
4,0.731611,0.084961,0.209243,0.041237,0.357908,0.000032,0.75,0.741210,1.000000,0.298565,0.804504,0.847991,0.0,0.547456,0


In [69]:
# X will be our examples and y will be our labels
X = final_data.drop('is_popular', axis=1)
y = final_data['is_popular']
# Sanity checks
print("Number of entries in actual data: " + str(len(X.index)))
print("Number of entries in label data: " + str(len(y.index)))

Number of entries in actual data: 570
Number of entries in label data: 570


In [70]:
# We drop this here because of high correlation with the our popularity metric
# - popular songs on Spotify also have a high youtube view count
X = X.drop('youtube_view_count', axis=1)

In [71]:
X.head()

,energy,liveness,tempo,speechiness,acousticness,instrumentalness,time_signature,danceability,key,duration,loudness,valence,mode
0,0.838293,0.331118,0.360693,0.399286,0.247690,0.000000,0.75,0.789030,0.636364,0.016514,0.807462,0.522258,1.0
1,0.502527,0.111965,0.431906,0.049366,0.574298,0.000000,0.75,0.825598,0.000000,0.442256,0.594084,0.444083,1.0
2,0.797866,0.028251,0.287019,0.083466,0.023312,0.000000,0.75,0.675105,0.090909,0.351029,0.627563,0.420195,1.0
3,0.779899,0.055152,0.756132,0.104877,0.048692,0.000000,0.75,0.586498,0.090909,0.395074,0.762286,0.868621,1.0
4,0.731611,0.084961,0.209243,0.041237,0.357908,0.000032,0.75,0.741210,1.000000,0.298565,0.804504,0.847991,0.0


# Wrapper methods

### Exhaustive search

In [10]:
import itertools
columns = ["energy", "liveness", "tempo", "speechiness", "acousticness", "instrumentalness", "time_signature", "danceability",
          "key", "duration", "loudness", "valence", "mode"]
allSubsets = []
total_no_subsets = 0
for m in range(1, len(columns) + 1, 1):
    subsets_with_m_elem = list(itertools.combinations(columns, m))
    total_no_subsets += len(subsets_with_m_elem)
    allSubsets.append(subsets_with_m_elem)
print(total_no_subsets)

8191


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

steps_taken = 0
scores_list = []

# Loop thorugh all the different subsets of 1,2,3, ..., (#features) elements
for n_elem_subsets in range(0, len(columns), 1):
    len_elem_subs = len(allSubsets[n_elem_subsets])
    best_score = 0
    feature_set = []
    # Loop through all the subsets of n_elem_subsets elements
    for cur_comb in range(0, len_elem_subs, 1):
        
        # Get the data with the current subset of features
        cur_data = X[list(allSubsets[n_elem_subsets][cur_comb])]
        
        # Transform to np array for faster computation
        cur_data_np = np.array(cur_data)
        
        # Counter of number of interations
        steps_taken += 1
        
        # Instantiate model
        # model = LogisticRegression(class_weight='balanced',random_state=4)
        model = svm.SVC(probability=False, gamma='scale', random_state=3, class_weight='balanced')
        
        # Instantiate cross validation strategy
        cv = StratifiedKFold(n_splits=10, random_state=3)
        
        # Compute the cross valdation scores
        scores = cross_val_score(model, cur_data_np, y, cv=cv, scoring='roc_auc')
        
        mean_score = scores.mean()
        
        # For brevity, we only keep the best feature subset of one size
        # Alternatively, we could keep the scores for all the different combinations
        if mean_score > best_score:
            best_score = mean_score
            feature_set = list(allSubsets[n_elem_subsets][cur_comb])
    #Keep the best score in a list    
    scores_list.append([feature_set, best_score])

print("Number of iterations: ", steps_taken)

Number of iterations:  8191


In [14]:
# Print feature sets with best scores
for cur_list in scores_list:
    print(cur_list)

[['valence'], 0.65573476052249635]
[['liveness', 'valence'], 0.67391872278664733]
[['liveness', 'tempo', 'valence'], 0.69570210449927428]
[['liveness', 'tempo', 'loudness', 'valence'], 0.70494920174165454]
[['energy', 'liveness', 'tempo', 'loudness', 'valence'], 0.70933599419448479]
[['energy', 'liveness', 'tempo', 'time_signature', 'loudness', 'valence'], 0.70625907111756159]
[['energy', 'liveness', 'tempo', 'acousticness', 'time_signature', 'loudness', 'valence'], 0.69929970972423805]
[['energy', 'liveness', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'loudness', 'valence'], 0.68986211901306249]
[['energy', 'liveness', 'tempo', 'acousticness', 'time_signature', 'danceability', 'loudness', 'valence', 'mode'], 0.68443940493468813]
[['energy', 'liveness', 'tempo', 'acousticness', 'time_signature', 'danceability', 'duration', 'loudness', 'valence', 'mode'], 0.68159107402031938]
[['energy', 'liveness', 'tempo', 'acousticness', 'instrumentalness', 'time_signature', 'danc

# Filter methods

### SelectKBest

In [61]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=5)

sel = SelectKBest(chi2, k = 6)
sel.fit(X_train, y_train)

selected = sel.transform(X_train)
#print(selected.shape)
#print(X_train.shape)

mask = sel.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, columns):
    if bool:
        new_features.append(feature)
print(new_features)

['tempo', 'acousticness', 'instrumentalness', 'key', 'valence', 'mode']


### Pearson correlation

In [54]:
import scipy
from scipy.stats import pearsonr

print(pearsonr(X['valence'], y))

(-0.14213373094378604, 0.00066591728398211357)


### Mutual information

In [81]:
from sklearn.feature_selection import mutual_info_classif

feature_scores = mutual_info_classif(X_train, y_train)

for score, fname in sorted(zip(feature_scores, columns)):
    print(fname, score)

danceability 0.0
key 0.0
loudness 0.0
time_signature 0.0
instrumentalness 0.00066222406065
duration 0.00394051170442
acousticness 0.00625966347405
valence 0.00993321327831
energy 0.0110091099907
liveness 0.011514378832
mode 0.016849549401
speechiness 0.0173146860389
tempo 0.023748408022


### Joint mutual information

In [93]:
columns = ["energy", "liveness", "tempo", "speechiness", "acousticness", "instrumentalness", "time_signature", "danceability",
          "key", "duration", "loudness", "valence", "mode"]
jmi = 0
for col in columns:
    for next_col in columns:
        if col != next_col:
            jmi += mutual_info_classif(X_train[[col, next_col]], y_train)[0]
            #print(col, next_col)
    print("JMI for {} is {}".format(col, jmi))

JMI for energy is 0.126908126194
JMI for liveness is 0.210939473696
JMI for tempo is 0.511648286241
JMI for speechiness is 0.679587761023
JMI for acousticness is 0.766574727049
JMI for instrumentalness is 0.850313356638
JMI for time_signature is 0.950601973209
JMI for danceability is 0.950601973209
JMI for key is 0.954111885769
JMI for duration is 1.00139802622
JMI for loudness is 1.00139802622
JMI for valence is 1.12912639673
JMI for mode is 1.19543325016


### ANOVA

In [75]:
from sklearn.feature_selection import f_classif

feature_scores = f_classif(X_train, y_train)[0]

for score, fname in sorted(zip(feature_scores, columns)):
    print(fname, score)

loudness 0.00568406128276
speechiness 0.0270260029669
time_signature 0.0319876660937
duration 0.185910425031
liveness 0.18892589418
mode 0.280688060472
instrumentalness 0.499371675555
key 0.824257236616
danceability 1.7323999922
acousticness 2.37542137751
energy 2.55239718842
tempo 2.90091388799
valence 9.47236752793


# Embedded methods

### LASSO regression

In [137]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=StratifiedKFold(n_splits=10, random_state=3), random_state=3)
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)

print("training score:", train_score)
print("test score: ", test_score)
print("number of features used: ", coeff_used)

#print(lasso.coef_)

ind = 0
sel_f = []
for coef in lasso.coef_:
    if coef != 0:
        sel_f.append(columns[ind]) 
    ind += 1
print("Alpha(amount of penalization) chosen by CV: ", lasso.alpha_)
print(sel_f)

training score: 0.0534300953141
test score:  -0.035451480573
number of features used:  4
Alpha(amount of penalization) chosen by CV:  0.00337342158855
['tempo', 'acousticness', 'danceability', 'valence']


In [143]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LogisticRegression
loglasso = LogisticRegression(penalty='l1', class_weight='balanced', random_state=3)
loglasso.fit(X_train,y_train)
train_score=loglasso.score(X_train,y_train)
test_score=loglasso.score(X_test,y_test)
coeff_used = np.sum(loglasso.coef_!=0)

print("training score:", train_score)
print("test score: ", test_score)
print("number of features used: ", coeff_used)

print(loglasso.coef_)

ind = 0
sel_f = []
for coef in loglasso.coef_[0]:
    if coef != 0:
        sel_f.append(columns[ind]) 
    ind += 1
print(sel_f)

training score: 0.694736842105
test score:  0.663157894737
number of features used:  9
[[-1.11084244  0.43384907 -1.24146497  0.         -0.7671785   0.          0.
   2.09339274 -0.02201686  0.          2.19290097 -3.60676226  0.05691074]]
['energy', 'liveness', 'tempo', 'acousticness', 'danceability', 'key', 'loudness', 'valence', 'mode']
